In [8]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate
from tensorflow.keras.models import Model

# Dados fictícios
data = {
    'usuario': [1, 2, 3, 4],
    'nacionalidade': ['BR', 'USA', 'USA', 'BR'],
    'estado': ['Estado1', 'Estado2', 'Estado3', 'Estado1'],
    'classe': ['prime', 'non prime', 'prime', 'prime'],
    'comprou': [[1, 3], [2], [1, 2], [3]]
}
df = pd.DataFrame(data)

# Criar mapeamentos de categorias
nacionalidades = df['nacionalidade'].unique()
estados = df['estado'].unique()
classes = df['classe'].unique()

nacionalidade_map = {nacionalidade: i for i, nacionalidade in enumerate(nacionalidades)}
estado_map = {estado: i for i, estado in enumerate(estados)}
classe_map = {classe: i for i, classe in enumerate(classes)}

# Adicionar colunas de mapeamento
df['nacionalidade_id'] = df['nacionalidade'].map(nacionalidade_map)
df['estado_id'] = df['estado'].map(estado_map)
df['classe_id'] = df['classe'].map(classe_map)

# Criar matriz de usuários e produtos
usuarios = df['usuario'].unique()
produtos = list(set(df['comprou'].explode().unique()))  
num_usuarios = len(usuarios)
num_produtos = len(produtos)
num_nacionalidades = len(nacionalidades)
num_estados = len(estados)
num_classes = len(classes)

# Criar matriz de interações usuário-produto
interacoes = np.zeros((num_usuarios, num_produtos))

for i, usuario in enumerate(usuarios):
    produtos_comprados = set(df[df['usuario'] == usuario]['comprou'].iloc[0])
    for j, produto in enumerate(produtos):
        if produto in produtos_comprados:
            interacoes[i, j] = 1

# Definir a arquitetura do modelo
input_dim = num_produtos

input_produto = Input(shape=(input_dim,))
embedding_produto = Embedding(input_dim, 16)(input_produto)
flatten_produto = Flatten()(embedding_produto)

input_nacionalidade = Input(shape=(1,))
embedding_nacionalidade = Embedding(num_nacionalidades, 4)(input_nacionalidade)
flatten_nacionalidade = Flatten()(embedding_nacionalidade)

input_estado = Input(shape=(1,))
embedding_estado = Embedding(num_estados, 4)(input_estado)
flatten_estado = Flatten()(embedding_estado)

input_classe = Input(shape=(1,))
embedding_classe = Embedding(num_classes, 4)(input_classe)
flatten_classe = Flatten()(embedding_classe)

concatenated_features = Concatenate()([flatten_produto, flatten_nacionalidade, flatten_estado, flatten_classe])
hidden_layer = Dense(16, activation='relu')(concatenated_features)
output_layer = Dense(input_dim, activation='sigmoid')(hidden_layer)

model = Model([input_produto, input_nacionalidade, input_estado, input_classe], output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Treinar o modelo com as interações usuário-produto
model.fit([interacoes, df['nacionalidade_id'], df['estado_id'], df['classe_id']], interacoes, epochs=50, batch_size=1, verbose=0)

# Gerar recomendações para todos os usuários
recomendacoes = []

for i in range(num_usuarios):
    interacao_usuario = interacoes[i].reshape(1, -1)
    nacionalidade_usuario = df[df['usuario'] == usuarios[i]]['nacionalidade_id'].iloc[0]
    estado_usuario = df[df['usuario'] == usuarios[i]]['estado_id'].iloc[0]
    classe_usuario = df[df['usuario'] == usuarios[i]]['classe_id'].iloc[0]

    recomendacao = model.predict([interacao_usuario, np.array([nacionalidade_usuario]), np.array([estado_usuario]), np.array([classe_usuario])])
    produtos_recomendados = sorted(list(enumerate(recomendacao[0])), key=lambda x: x[1], reverse=True)[:2]
    produtos_recomendados = [produtos[idx] for idx, _ in produtos_recomendados]
    recomendacoes.append(produtos_recomendados)


1/1 [==============================] - 0s 22ms/step


In [10]:

# Adicionar coluna de recomendações ao dataframe original
df['recomendacoes'] = recomendacoes

df

,usuario,nacionalidade,estado,classe,comprou,nacionalidade_id,estado_id,classe_id,recomendacoes
0,1,BR,Estado1,prime,"[1, 3]",0,0,0,"[3, 1]"
1,2,USA,Estado2,non prime,[2],1,1,1,"[2, 3]"
2,3,USA,Estado3,prime,"[1, 2]",1,2,0,"[2, 1]"
3,4,BR,Estado1,prime,[3],0,0,0,"[3, 1]"


In [6]:
# Função para remover elementos da coluna "comprou" da coluna "recomendacoes"
def remove_comprados(row):
    comprados = set(row['comprou'])
    recomendacoes = [produto for produto in row['recomendacoes'] if produto not in comprados]
    return recomendacoes


In [12]:
# Aplicar a função em cada linha do dataframe
df['recomendacoes'] = df.apply(remove_comprados, axis=1)

df

,usuario,nacionalidade,estado,classe,comprou,nacionalidade_id,estado_id,classe_id,recomendacoes
0,1,BR,Estado1,prime,"[1, 3]",0,0,0,[]
1,2,USA,Estado2,non prime,[2],1,1,1,[3]
2,3,USA,Estado3,prime,"[1, 2]",1,2,0,[]
3,4,BR,Estado1,prime,[3],0,0,0,[1]


In [4]:
import pandas as pd

# Exemplo de dataframe com colunas "comprou" e "recomendacoes"
data = {
    'comprou': [[1, 3], [2], [1, 2], [3]],
    'recomendacoes': [[1, 2, 3, 4], [2, 3, 5, 6], [1, 3, 4, 6], [2, 4, 5, 6]]
}
df = pd.DataFrame(data)

df 

,comprou,recomendacoes
0,"[1, 3]","[1, 2, 3, 4]"
1,[2],"[2, 3, 5, 6]"
2,"[1, 2]","[1, 3, 4, 6]"
3,[3],"[2, 4, 5, 6]"


  comprou recomendacoes
0  [1, 3]        [2, 4]
1     [2]     [3, 5, 6]
2  [1, 2]     [3, 4, 6]
3     [3]  [2, 4, 5, 6]
